## IDEA <br>

- Use OMDB API: http://www.omdbapi.com/#usage
- Do the API request with the search, movies with word 'love', 2 pages
- Use this URL with parameters: http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie
- Like that get the list of titles of the movies (later create column in df)
- Do another API request title by title to get more info about the movies
- Use this URL with parameters: http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t=The+Shawshank+Redemption
- Check how to pass the movie name to the end of URL. Format: xx+xx+xxx+xx (words separated by '+')
- Put info from both requests to the df
- Scrap IMDB or Wikkipedia based on title or IMDB ID to get info from there
- Check if some data matching: Release year, Genre, Director, IMDB Rating
- Could also get more data about the director from Wiki or IMDB

## API <br>

- For the API portion of the project will need to make calls to your chosen API, successfully obtain a response, request data, convert it into a Pandas data frame, and export it as a CSV file.

## WEB SCRAPING <br>

- For the web scraping portion of the project, you will need to scrape the HTML from your chosen page, parse the HTML to extract the necessary information, and either save the results to a text (txt) file if it is text or into a CSV file if it is tabular data.

## 1. API

In [10]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

#check if using this library
import time

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#the function to flatten_json several times
#for now haven't used them

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

- Doing the search of the titles that include word 'love' in 2 pages

In [66]:
url= 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie'
response = requests.get(url)
response

results = response.json()
results

<Response [200]>

{'Search': [{'Title': 'Crazy, Stupid, Love.',
   'Year': '2011',
   'imdbID': 'tt1570728',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg2MjkwMTM0NF5BMl5BanBnXkFtZTcwMzc4NDg2NQ@@._V1_SX300.jpg'},
  {'Title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb',
   'Year': '1964',
   'imdbID': 'tt0057012',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BZWI3ZTMxNjctMjdlNS00NmUwLWFiM2YtZDUyY2I3N2MxYTE0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg'},
  {'Title': 'Love, Actually',
   'Year': '2003',
   'imdbID': 'tt0314331',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BMTY4NjQ5NDc0Nl5BMl5BanBnXkFtZTYwNjk5NDM3._V1_SX300.jpg'},
  {'Title': 'Shakespeare in Love',
   'Year': '1998',
   'imdbID': 'tt0138097',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BM2ZkNjM5MjEtNTBlMC00OTI5LTgyYmEtZDljMzNmNzhiNzY0XkEyXkFqcGdeQXVyNDYyMDk5MTU@._V1_SX300.jpg'},
  {'Title': 'P.S. I L

- creating the list of titles with the word 'love'

In [67]:
#API gives max 10 movies

title_box = results.get('Search')
title_list = [title.get('Title') for title in title_box]
title_list

['Crazy, Stupid, Love.',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb',
 'Love, Actually',
 'Shakespeare in Love',
 'P.S. I Love You',
 'I Love You, Man',
 'Love & Other Drugs',
 'Punch-Drunk Love',
 'Love, Rosie',
 'In the Mood for Love']

- converting names into the correct format to later pass them to the URL

In [68]:
titles_formatted = [string.replace(" ", "+") for string in title_list]
titles_formatted

['Crazy,+Stupid,+Love.',
 'Dr.+Strangelove+or:+How+I+Learned+to+Stop+Worrying+and+Love+the+Bomb',
 'Love,+Actually',
 'Shakespeare+in+Love',
 'P.S.+I+Love+You',
 'I+Love+You,+Man',
 'Love+&+Other+Drugs',
 'Punch-Drunk+Love',
 'Love,+Rosie',
 'In+the+Mood+for+Love']

- Getting the info about each movie from the list

In [69]:
url_pattern = 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t='
appended_dfs=[]

for title in titles_formatted:
    url = url_pattern + title
    response = requests.get(url)
    results = response.json()
    flattened_data = json_normalize(results)
    #appending dataframes to the list to later concatenate them to one dataframe
    appended_dfs.append(flattened_data)
    love_movies = pd.concat(appended_dfs)

love_movies

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,"Crazy, Stupid, Love.",2011,PG-13,29 Jul 2011,118 min,"Comedy, Drama, Romance","Glenn Ficarra, John Requa",Dan Fogelman,"Steve Carell, Ryan Gosling, Julianne Moore, Em...",A middle-aged husband's life changes dramatica...,...,68,7.4,"447,664",tt1570728,movie,01 Nov 2011,"$83,300,000",Warner Bros. Pictures,N/A,True
0,Dr. Strangelove or: How I Learned to Stop Worr...,1964,PG,29 Jan 1964,95 min,Comedy,Stanley Kubrick,"Stanley Kubrick (screenplay), Terry Southern (...","Peter Sellers, George C. Scott, Sterling Hayde...",An insane general triggers a path to nuclear h...,...,97,8.4,"427,212",tt0057012,movie,02 Nov 2004,N/A,Sony Pictures,N/A,True
0,"Love, Actually",2003,R,14 Nov 2003,135 min,"Comedy, Drama, Romance",Richard Curtis,Richard Curtis,"Bill Nighy, Gregor Fisher, Rory MacGregor, Col...",Follows the lives of eight very different coup...,...,55,7.6,"405,898",tt0314331,movie,27 Apr 2004,"$59,365,105",Universal Pictures,N/A,True
0,Shakespeare in Love,1998,R,08 Jan 1999,123 min,"Comedy, Drama, History, Romance",John Madden,"Marc Norman, Tom Stoppard","Geoffrey Rush, Tom Wilkinson, Steven O'Donnell...","The world's greatest ever playwright, William ...",...,87,7.1,"205,961",tt0138097,movie,10 Aug 1999,N/A,Miramax,N/A,True
0,P.S. I Love You,2007,PG-13,21 Dec 2007,126 min,"Comedy, Drama, Romance",Richard LaGravenese,"Richard LaGravenese (screenplay), Steven Roger...","Hilary Swank, Gerard Butler, Lisa Kudrow, Gina...",A young widow discovers that her late husband ...,...,39,7.0,"194,541",tt0431308,movie,05 Feb 2008,N/A,N/A,N/A,True
0,"I Love You, Man",2009,R,20 Mar 2009,105 min,"Comedy, Romance",John Hamburg,"John Hamburg (screenplay), Larry Levin (screen...","Paul Rudd, Rashida Jones, Sarah Burns, Greg Le...",Friendless Peter Klaven goes on a series of ma...,...,70,7.0,"188,613",tt1155056,movie,11 Aug 2009,"$71,300,000",Dreamworks/Paramount,N/A,True
0,Love,2015,Not Rated,30 Oct 2015,135 min,"Drama, Romance",Gaspar Noé,Gaspar Noé,"Aomi Muyock, Karl Glusman, Klara Kristin, Ugo Fox",Murphy is an American living in Paris who ente...,...,51,6.1,"40,472",tt3774694,movie,15 Mar 2016,"$176,061",Alchemy,N/A,True
0,Punch-Drunk Love,2002,R,01 Nov 2002,95 min,"Comedy, Drama, Romance, Thriller",Paul Thomas Anderson,Paul Thomas Anderson,"Adam Sandler, Jason Andrews, Don McManus, Emil...",A psychologically troubled novelty supplier is...,...,78,7.3,"137,629",tt0272338,movie,24 Jun 2003,"$17,800,000",Columbia Pictures,N/A,True
0,"Love, Rosie",2014,R,22 Oct 2014,102 min,"Comedy, Romance",Christian Ditter,"Juliette Towhidi (screenplay), Cecelia Ahern (...","Lily Collins, Sam Claflin, Christian Cooke, Ja...",Rosie and Alex have been best friends since th...,...,44,7.2,"111,376",tt1638002,movie,05 May 2015,N/A,Film Arcade,N/A,True
0,In the Mood for Love,2000,PG,09 Mar 2001,98 min,"Drama, Romance",Kar-Wai Wong,Kar-Wai Wong,"Maggie Cheung, Tony Chiu-Wai Leung, Ping Lam S...","Two neighbors, a woman and a man, form a stron...",...,85,8.1,"111,148",tt0118694,movie,05 Mar 2002,N/A,USA Films,N/A,True
